# Improved ICU Stay and Mortality Prediction via Feature Engineering of Vital Sign Data

### Dylan Mack and Julia Tompkins

GitHub Link: https://github.com/juliatompkins/surgical_critical_event_prediction

# Executive Summary

The operating room is a data-rich environment in which analytical frameworks offer great
potential for improving health outcomes and reducing costs through improved decision-making. Both patients and surgeons must make decisions about whether to proceed with a surgery, which involves weighing risks against potential benefits. Additionally, as patients recover, hospitals must allocate resources to these patients in proportion to the intensity of their recovery process. Thus, our workflow aims to address these challenges through two prediction tasks. First, we will predict the likelihood of mortality for a patient during/following their surgery, which will help doctors and patients determine if a surgery is worth doing. Second, we will predict the number of days a patient will spend recovering in the ICU, which will help hospitals determine the volume of supplies they are likely to require over a given window. The unique contribution of our project will be the integration between clinical patient information features and biosignal (time series) features in our workflow. This project uses the **vitaldb** dataset, which contains clinical and biosignal data from 6,388 non-cardiac surgical patients across 10 different operating rooms in Seoul National University Hospital, Seoul, Republic of Korea.

# Questions and Answers

1. How would you like to aggregate your data for time-series, hourly, daily, or monthly?
2. As your data is time series, will you be aggregating the data? Since it is a time series model you are making, how do you select the time interval for your data?

Our time series data is from ECG measurements. The data is taken at a rate of 500 Hz and only encompasses the length of one surgery, which typically ranges from less than an hour to a few hours, so hourly/daily/monthly aggregations would not make sense in our context. We use the entire time interval with the first and last 15 minutes removed. In addition, we are only using the time series data for feature construction and not directly for prediction. 

3. What are you going to do with highly correlated variables?

Depending on the age strata, we do have some highly correlated variables, like height and weight in children. We will use nested, repeated cross-validation to aid in our process of feature selection. From this cross-validation, using pycaret, we can observe which combination of features gives the optimal performance. Similarly, pycaret allows us to produce a feature importance plot, illustrating how significant each feature is in predicting outcomes. These methods will allow us to determine which combinations of highly correlated variables to retain as features.

4. How did you determine the threshold for identifying an anomalous heart rate during surgery?

In one of our potential feature construction methods, we refer to a ‘threshold’, where our ultimate feature is the percentage of R-R intervals that fall outside the threshold. Our threshold was determined to be anywhere outside of two standard deviations of the mean of the R-R intervals, which was inspired by traditional outlier detection methods. We know that this isn’t the perfect approach, as ‘outlier’ or ‘anomalous’ heart rates could also drag the mean in certain directions, but we’re still working on how to handle that part.

5. You mentioned using lCU stay duration as a binary classification problem. What factors led you to focus on this target instead of in-hospital mortality?

We were originally looking at in-hospital mortality at our target variable, but there was an extreme class imbalance, with less than 1% of patients dying in the hospital. We have learned many methods to handle class imbalance, but this seemed like a very severe class imbalance. Pivoting our problem, we reduced our sample space to only include patients with some ICU stay (which was still over 1,000 instances) and made a new binary classification problem, where a short ICU stay was one day and a long ICU stay was longer than one day. We still have a class imbalance, but now it is about 70-30, which felt far more manageable.

6. I see some Fourier transform in your analysis...would you be training in frequency or time domain?

All of our time series analysis is used to create a new feature to add back into our original model, which uses static patient and surgery information data. We are not training directly on time series data, so we are not really training in the frequency or time domain. The place where you saw Fourier analysis was in our exploration of different methods of constructing a feature from time series ECG data, but we ultimately decided not to continue with that approach.

7. Why do you think big jumps are a good feature to represent your time series data?

There is a subtle but important distinction to make here. We do not think jumps represent the time series data as a whole; instead, we think jumps measure large changes in heart rate, which may correspond to adverse health outcomes. If a patient’s heartbeat is rapidly oscillating between high BPM and low BPM, this may be associated with stress on the heart, which could in turn lead to worse ICU outcomes. This is why we have opted to include jumps as a feature for our classifier.

8. As we observe that correlation matrix of children distinct from the ones of the other two groups, will you handle these two groups differently in your prediction model

Yes - this is the core of our stratification strategy. We will handle these two groups differently by training separate models for children, adults, and elders. This will allow our models to learn the patterns that are specific to each developmental age group. 

9. What would you say the biggest trade-off that you had to make in terms of model complexity or interpretability?

At the time of this report, we had no models to share. Now that we are working on models, we have more insight. We initially started with incredibly interpretable models like logistic regression, but found poor performance. Now, we are looking at more complex models like gradient boosting classifiers, but we are making sure to keep in mind the fact that complex models can often overfit to the training data. This is something we’re still working on!

10. I'd love to hear more about why you think the FFT method failed -- it seems like the RR interval method is based on a similar notion of decomposing signals, so some form of Fourier analysis should be fruitful, no?

This is a great question and maybe something we didn’t communicate perfectly in our presentation. The FFT method did not necessarily fail, but it did not produce as fruitful of results as our RR analysis. Essentially, we realized that the FFT was too generic of a tool to be ideal and we could deploy methods that are more specifically suited to our problem. In particular, instead of doing a wave decomposition into a fourier series, we could instead do a wave decomposition that fits the specific QRS waves that are known to be present in EEG data. This approach was more tailored to our problem domain and yielded better insights.

11. What is the split between number of observations of children vs adults? Is there a possible imbalance due to the types of surgeries being performed on two different populations? I would love to know more about why in the unsegmented data, children were flagged as outliers.

Our initial dataset (before we filtered to only include ICU stays) included 74 children, 4,165 adults, and 2,149 elders. We do believe that children were flagged as outliers because of this imbalance. Since they tend to have lower height and weight, for example, and represent a minority class, children were being identified as anomalous as a whole, rather than anomalous individuals within each developmental group.

12. What method did you use to stratify the age ranges within your dataset? I know train_test_split does a good job of doing it, but there are other various kinds of options that one may be more inclined to use based on their specific scenario (K-Fold, SMOTE, Stratification of Multiple Columns, Stratification based on percentiles, etc...).

Rather than using a method from data analytics to set the specific threshold on age strata, we instead relied on domain-specific subject matter knowledge. The key insight here is that surgery outcomes, as well as certain physical features, are highly sensitive to developmental age. For example, there is a strong correlation between age, height, and weight as a child, but this correlation diminishes as an adult and elder. Similarly, surgery is much riskier for children and elders than adults. Thus, the stratification we implemented is based on known developmental differences, rather than a method like K-Fold or SMOTE. 